In [1]:
!pip install transformers &> null
!pip install flask_ngrok &> null
!pip install flask-cors  &> null

from flask import Flask, jsonify
from flask_ngrok import run_with_ngrok
from flask import request
from flask_cors import CORS, cross_origin
import warnings
warnings.filterwarnings('ignore')



import json
import torch
import string
from transformers import RobertaForTokenClassification, RobertaTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

bert_version = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(bert_version)
bert_model_loc = '/content/drive/Shareddrives/SWM - NER/models/BERT/BERT-6-epochs.pth'
bert_model = torch.load(bert_model_loc)

label2id = {'0': 'O',
            '1': 'B-PER',
            '2': 'I-PER',
            '3': 'B-ORG',
            '4': 'I-ORG',
            '5': 'B-LOC',
            '6': 'I-LOC',
            '7': 'B-MISC',
            '8': 'I-MISC'}


def get_named_entities(data):
    test_string = data.translate(str.maketrans('', '', string.punctuation)).split(" ")
    test_sample = [test_string]
    sample = tokenizer(test_sample, truncation=True, padding='max_length', is_split_into_words=True)
    for k, v in sample.items():
        sample[k] = torch.tensor(sample[k])

    batch = { k: v.to(device) for k, v in sample.items() }
    outputs = bert_model(**batch)

    named_entities = list()
    pred_classes = map(int, torch.argmax(outputs.logits, dim=2)[0][:len(test_string)])
    for word, pred in zip(test_string, pred_classes):
        if str(pred) == "0":
            continue
        named_entities.append([word, label2id[str(pred)]])
    return named_entities


In [4]:
# data = "New York City, often called simply New York, is the most populous city in the United States. With an estimated 2019 population of 8,336,817 distributed over about 302.6 square miles, New York City is also the most densely populated major city in the United States."
# print(get_named_entities(data))


In [ ]:
from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import request

app = Flask(__name__)
run_with_ngrok(app)  
CORS(app)


@app.route('/', methods=['GET', 'POST'])
@cross_origin()
def run_app():
    return "/pred_bert/ for Predictions"


@app.route('/pred_bert/', methods=['GET', 'POST'])
def pred_app():        
    jsonData = request.get_json(force=True)    

    result = dict()
    result["data"] = get_named_entities(jsonData["data"])
    result["count"] = len(result["data"])
    return json.dumps(result)


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6345c0b4776b.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [06/Apr/2021 02:42:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 02:42:40] "GET /pred_bert/ HTTP/1.1" 400 -
127.0.0.1 - - [06/Apr/2021 02:45:40] "OPTIONS /pred_bert/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 02:45:40] "POST /pred_bert/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 02:55:07] "OPTIONS /pred_bert/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 02:55:07] "POST /pred_bert/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 02:55:48] "OPTIONS /pred_bert/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 02:55:48] "POST /pred_bert/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 02:58:23] "OPTIONS /pred_bert/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 02:58:23] "POST /pred_bert/ HTTP/1.1" 200 -
